In [1]:
!pip install emoji opencc-python-reimplemented


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 31.8 MB/s eta 0:00:00


In [2]:
# Phase 1.2｜周杰倫粉絲留言資料清洗（留言語料前處理）
!pip install emoji opencc-python-reimplemented -q

import pandas as pd
import re
import emoji
from opencc import OpenCC

# === 1. 讀取原始留言資料 CSV ===
df = pd.read_csv("jay_playlist_comments.csv")  # <--- 你上傳的留言檔案名

# === 2. 移除空值與重複留言 ===
df = df.dropna(subset=["text"]).drop_duplicates(subset=["text"]).reset_index(drop=True)

# === 3. 清理函數（emoji、網址、空白）===
def clean_text(text):
    # 去除 emoji
    text = emoji.replace_emoji(text, replace='')
    # 去除網址
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # 去除空格與換行
    text = text.replace('\n', '').replace('\r', '').strip()
    return text

df["text_cleaned"] = df["text"].astype(str).apply(clean_text)

# === 4. 簡體轉繁體 ===
converter = OpenCC('s2t')  # s2t: Simplified to Traditional
df["text_cleaned"] = df["text_cleaned"].apply(converter.convert)

# === 5. 移除非中文留言（保留含有至少2個中文字的）===
def contains_chinese(text):
    return len(re.findall(r'[\u4e00-\u9fff]', text)) >= 2

df = df[df["text_cleaned"].apply(contains_chinese)].reset_index(drop=True)

# === 6. 輸出清洗後的語料 CSV ===
df.to_csv("jay_comments_clean.csv", index=False, encoding="utf-8-sig")
print(f"✅ 清洗完成，總共 {len(df)} 筆留言已儲存至 jay_comments_clean.csv")

# === 7. 顯示前幾筆供檢查 ===
df[["text", "text_cleaned"]].head(10)


✅ 清洗完成，總共 58315 筆留言已儲存至 jay_comments_clean.csv


,text,text_cleaned
0,青春时光机来一首可爱女人🌈,青春時光機來一首可愛女人
1,徐若瑄那時期很正很漂亮,徐若瑄那時期很正很漂亮
2,初中時是用這首歌追到老師的 老師的孩子現在也應該是上初中了,初中時是用這首歌追到老師的 老師的孩子現在也應該是上初中了
3,没想到你对我念念不忘，跟你身边的美女不能比！,沒想到你對我念念不忘，跟你身邊的美女不能比！
4,徐若瑄那個時候真的美到驚心動魄,徐若瑄那個時候真的美到驚心動魄
5,为什么二十年前的歌还那么好听。天哪,爲什麼二十年前的歌還那麼好聽。天哪
6,今でもこの曲が大好き!!\n思い出の曲です…,今でもこの曲が大好き!!思い出の曲です…
7,所以才會有今生一見如故的感覺,所以才會有今生一見如故的感覺
8,徐若瑄學姊簽到😏🙂🙃😉,徐若瑄學姊簽到
9,2025年還在聽😂,2025年還在聽
